# IEX Cloud API Access samples

Based on the course [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) by Nick McCullum from Youtube channel [freeCodeCamp.org](https://www.youtube.com/channel/UC8butISFwT-Wl7EV0hUK0BQ).

* Main site: https://iexcloud.io/
* Documentation: https://iexcloud.io/docs/api/

In [1]:
import numpy as np
import pandas as pd
import requests

## Acquiring an API Token

Create account in https://iexcloud.io/ Tokens are available in https://iexcloud.io/console/tokens

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get published to your local Git repository.

In [2]:
from secrets import IEX_CLOUD_API_TOKEN

Set a list of some stocks

In [3]:
stocks = ['AMZN','AAPL','GOOG','MSFT','NFLX','V','PYPL','MA','ADBE','INTC','CRM','AVGO',
          'ACN','TXN','QCOM','ORCL','INTU','AMAT','AMD','NOW','SQ','MU','FB','LRCX','ADP']

Note than in [Algorithmic Trading Using Python - Full Course](https://www.youtube.com/watch?v=xfzGZB4HhEE) the `https://sandbox.iexapis.com/` is used. It provides simulated test data.

However for real market data the `https://cloud.iexapis.com/` should be used. Note that some of the functions may require payment. Check details in your account.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of the code.

This is because HTTP requests are typically one of the slowest components of a scripts.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement in this section, we'll split our list of stocks of 100 and then make a batch API call for each group.

In [4]:
symbol_strings = ','.join(stocks)
symbol_strings

'AMZN,AAPL,GOOG,MSFT,NFLX,V,PYPL,MA,ADBE,INTC,CRM,AVGO,ACN,TXN,QCOM,ORCL,INTU,AMAT,AMD,NOW,SQ,MU,FB,LRCX,ADP'

In [5]:
dataframe_columns = ['Tickers', 'PE_Ratio', 'Current_Price', 'Earnings_Per_Share']

In [6]:
batch_dataframe = pd.DataFrame(columns=dataframe_columns)

batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_strings}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'

data = requests.get(batch_api_call_url).json()
for symbol in symbol_strings.split(','):
    batch_dataframe = batch_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['peRatio'],
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['ttmEPS']
            ],
            index = dataframe_columns
        ),
        ignore_index=True
    )

In [7]:
batch_dataframe

,Tickers,PE_Ratio,Current_Price,Earnings_Per_Share
0,AMZN,57.57,3303.50,57.3800
1,AAPL,29.19,148.89,5.1000
2,GOOG,NaN,2767.79,0.0000
3,MSFT,36.00,289.81,8.0500
4,NFLX,52.92,510.72,9.6500
5,V,46.83,231.79,4.9500
6,PYPL,66.97,274.59,4.1000
7,MA,50.27,361.98,7.2000
8,ADBE,54.87,634.35,11.5600
9,INTC,11.92,53.54,4.4900


In [8]:
batch_dataframe.to_csv('InformationTechnologyStocks.csv', index = False)